# Palabras Relacionadas - Dataset

En el presente notebook se documenta el preprocesamiento y creación del dataset del proyecto.
## Preprocesamiento documentos
- **Documentos a procesar:** son pertenecientes a las distintas materias, según lo especificado en el [plan de Ingeniería en Sistemas de Información 2023, UTN - FRM](https://www.lamanuelsavio.org/wp-content/uploads/2024/02/Plan-Sistemas.pdf).
- **Ubicación de los documentos a procesar:** desde el root del proyecto `/data/raw`.
- **Ubicación de los documentos procesados:** desde el root en `/data/plain`
- **Nombre de los documentos:** consta de la siguiente sintaxis `<nivel materia> - <abreviatura materia> - <título del material>`
    - `<nivel materia>`: año (entero) de cursado según plan de estudios. Ej. `5` para quinto año
    - `<abreviatura materia>`: abreviatura del nombre de la materia. Ej. `SO` para Sistemas Operativos
    - `<título del material>`: nombre original significativo del pdf

Los **documentos a procesar están en formato .pdf**, por lo que se utiliza la librería [pypdf](https://pypdf.readthedocs.io/en/stable/).

In [2]:
!pip install pymupdf

   ---------------------------------------- 0.0/18.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/18.5 MB 16.4 MB/s eta 0:00:02
   ------- -------------------------------- 3.7/18.5 MB 18.1 MB/s eta 0:00:01
   ------------------ --------------------- 8.4/18.5 MB 18.6 MB/s eta 0:00:01
   --------------------------- ------------ 12.8/18.5 MB 19.6 MB/s eta 0:00:01
   ------------------------------------- -- 17.6/18.5 MB 20.1 MB/s eta 0:00:01
   ---------------------------------------- 18.5/18.5 MB 19.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import fitz
import os
from os import listdir
from os.path import isfile, join

# Seteo el path al root del proyecto
dev_folder = 'dev'
if(os.getcwd().split('/')[-1] == dev_folder):
    os.chdir('../')
print(os.getcwd()) # debugging, debe imprimir el path al root del proyecto

c:\Users\saoga\OneDrive\Escritorio\Repos\TPI-RNP-Palabras-Relacionadas-ISI\dev


Creamos el método `raw_to_plain` que dados:
- `raw_path`: path de los archivos a procesar
- `plain_path`: path de los archivos procesados 
 
Crea un archivo de texto plano (`.txt`) por cada archivo procesado y lo guarda en el `plain_path`.

In [5]:
def raw_to_plain(raw_path, plain_path):
    raw_files = [os.path.splitext(f)[0] for f in listdir(raw_path) if isfile(join(raw_path, f))]
    
    for file in raw_files:
        print("\033[94mConvirtiendo archivo: " + file + "\033[0m")
        
        try:
            doc = fitz.open(f'{raw_path}/{file}.pdf')
            
            with open(f'{plain_path}/{file}.txt', 'w', encoding='utf-8') as plain_file:
                for page in doc:
                    text = page.get_text("text") 
                    plain_file.write(text)
                    
            doc.close()
            
        except Exception as e:
            print(f"\033[91mError processing {file}: {str(e)}\033[0m")


In [8]:
raw_path = "./data/raw"
plain_path = "./data/plain"

raw_to_plain(raw_path, plain_path)

Convirtiendo archivo: 4 - AS - Implementación de un Data Center
Convirtiendo archivo: 4 - RD - Kurose-Ross
Convirtiendo archivo: 5 - SSI - guia_ciberseguridad_gestion_riesgos_metad
Convirtiendo archivo: 4 - AS - Sniffers_y_escaneo_de_puertos
Convirtiendo archivo: 4 - AS - Backups_raids
Convirtiendo archivo: 4 - IO - Breve Resumen ANALISIS SENSIBILIDAD
Convirtiendo archivo: 1 - SyO - Resumen SyO U5
Convirtiendo archivo: 5 - SSI - Magerit_v3_libro1_metodo
Convirtiendo archivo: 3 - AP - AdProy_2_Trabajo en Equipo_2022
Convirtiendo archivo: 5 - SSI - DE641-2021_Anexo
Convirtiendo archivo: 1 - SyO - 2)Recopilacion de la informacion
Convirtiendo archivo: 1 - SyO - 05 Arquitectura_empresarial_que_es_y_para_q
Convirtiendo archivo: 3 - CD - sistemas-de-comunicaciones-electronicas-tomasi-4ta-edicion
Convirtiendo archivo: 5 - SSI - Resumen U2 - Disposición ONTI 1 2015
Convirtiendo archivo: 3 - BD - caselli_manual-de-base-de-datos-2019
Convirtiendo archivo: 2 - SO - Stallings
Convirtiendo archivo:

In [9]:
def get_metrics(plain_path, split_char='-'):
    metrics = {}

    plain_files = [os.path.splitext(f)[0] for f in listdir(plain_path) if isfile(join(plain_path, f))]

    for file in plain_files:

        sf = file.split(split_char)
        anio = sf[0].strip()
        materia = sf[1].strip()

        if(not anio in metrics):
            metrics[anio] = {}
        if(not materia in metrics[anio]):
            metrics[anio][materia] = 0
        metrics[anio][materia] += os.path.getsize(f'{plain_path}/{file}.txt')
    
    return metrics

def print_metrics(metrics):
    for anio, materias in metrics.items():
        print(f"Año {anio}:")
        for materia, tamano in materias.items():
            print(f"\tMateria: {materia} - {str(round(tamano/1000000,2))}MB")
            # TODO: tal vez mostrar la sumatoria de los tamaños
            
print_metrics(get_metrics(plain_path))

Año 1:
	Materia: AyED - 0.07MB
	Materia: SyO - 1.16MB
	Materia: MD - 0.71MB
	Materia: AC - 0.44MB
Año 4:
	Materia: ICS - 0.24MB
	Materia: AS - 2.36MB
	Materia: TA - 0.06MB
	Materia: UXUI - 0.11MB
	Materia: IO - 0.03MB
	Materia: RD - 5.17MB
	Materia: S - 0.02MB
Año 3:
	Materia: DS - 1.28MB
	Materia: BD - 0.26MB
	Materia: AP - 0.14MB
	Materia: CD - 4.85MB
Año 2:
	Materia: SO - 5.8MB
	Materia: AS - 0.59MB
Año 5:
	Materia: SSI - 1.47MB
	Materia: GG - 0.01MB


## Tokenización
A continuación se tokenizan los documentos procesados planos `.txt`, generando un nuevo archivo por cada documento procesado fuente, donde cada línea representa un token. 

Para ello se utiliza la librería [spacy](https://spacy.io/), la cual tomará un rol importante para obtención de tokens, filtrado y transformación de los mismos.

In [4]:
!pip install spacy
!python -m spacy download es_core_news_sm


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     ----- ---------------------------------- 1.8/12.9 MB 12.6 MB/s eta 0:00:01
     --------------------- ------------------ 6.8/12.9 MB 19.1 MB/s eta 0:00:01
     ------------------------------- ------- 10.5/12.9 MB 19.3 MB/s eta 0:00:01
     --------------------------------------  12.8/12.9 MB 16.8 MB/s eta 0:00:01
     --------------------------------------  12.8/12.9 MB 16.8 MB/s eta 0:00:01
     --------------------------------------  12.8/12.9 MB 16.8 MB/s eta 0:00:01
     --------------------------------------  12.8/12.9 MB 16.8 MB/s eta 0:00:01
     --------------------------------------  12.8/12.9 MB 16.8 MB/s eta 0:00:01
     --------------------------------------  12.8/12.9 MB 16.8 MB/s eta 0:00:01
     --------------------------------------  12.8/12.9 MB 16.8 MB/s eta 0:00:01
     --------------------------------------  12.8/12.9 M


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import spacy
import re

esp = spacy.load("es_core_news_sm")

In [2]:
def tokenize_files(plain_path, tokens_path, banned_tokens):
    raw_files = [os.path.splitext(f)[0] for f in listdir(plain_path) if isfile(join(plain_path, f))]

    def is_clean_token(token):
        return not (
            token.is_punct or
            token.is_space or
            token.is_stop or
            len(token.text) == 1 or
            token.text in banned_tokens or 
            bool(re.search(r'(^[0-9\.\,]+$)|(-$)|(^.\.$)', token.text)))

    for f in raw_files:
        print("\033[94mTokenizando archivo: " + f + "\033[0m")

        with open(f"{plain_path}/{f}.txt", "rb") as pf:
            txt = pf.read().decode("utf-8")
            tokens = esp.tokenizer(txt)
            with open(f"{tokens_path}/{f}.txt", "wb") as tf:
                for token in tokens:
                    if (is_clean_token(token)):
                        tf.write((token.text + "\n").encode("utf-8"))

In [10]:
tokens_path = "./data/tokens"
banned_tokens = set(["capítulo", "página", "figura", "cap", "ejemplo", "catedra", "mendoza", "argentina", "muñoz", "facchini", "cesari", "xsd", "infoleg"])
tokenize_files(plain_path, tokens_path, banned_tokens)

Tokenizando archivo: 1 - AyED - Unidades 1 y 2 (cód. fotoc. 7928)
Tokenizando archivo: 1 - SyO - Resumen SyO U3
Tokenizando archivo: 4 - ICS - 2.-principiosingenieriasoftware
Tokenizando archivo: 3 - DS - MerFNConceptos
Tokenizando archivo: 2 - SO - ResumenSO
Tokenizando archivo: 1 - AyED - Unidad4 (7930)
Tokenizando archivo: 2 - AS - U8 Metodologías Agiles parte a 
Tokenizando archivo: 3 - DS - Libro UML y Patrones - Larman
Tokenizando archivo: 4 - AS - Teletrabajo-4k9
Tokenizando archivo: 4 - ICS - SoftwareDesign_PrincipiosyPatrones-Autentia
Tokenizando archivo: 4 - TA - Guía 1. Sistemas de control automático
Tokenizando archivo: 1 - SyO - 04 gestion-por-procesos
Tokenizando archivo: 1 - AyED - Unidad3 (7929)
Tokenizando archivo: 4 - AS - Gestión CPD
Tokenizando archivo: 5 - SSI - M5-Privacidad
Tokenizando archivo: 3 - BD - caselli_manual-de-base-de-datos-2019
Tokenizando archivo: 4 - UXUI - Diseño UX UNIDAD 1
Tokenizando archivo: 4 - AS - Teletrabajo-4k10
Tokenizando archivo: 4 - AS

### Tokenización con ventana deslizante
Una vez que se tienen los archivos con los tokens, deseamos **detectar conceptos adicionales**. Aquellos conceptos que adquieren significado con la combinación de palabras.

Para ello se utilizará un **algoritmo de ventana deslizante** con un `window_size = 4`. Lo que buscamos es dada una secuencia de tokens `["sistemas", "operativos", "distribuídos"]`formar los siguientes conceptos: `"sistemas operativos"`, `"sistemas operativos distribuídos"`.

In [6]:
from itertools import combinations
import re

window_size = 4

In [51]:
related_table = {}
raw_files = [os.path.splitext(f)[0] for f in listdir(plain_path) if isfile(join(plain_path, f))]
banned = ["capítulo", "página", "figura", "cap", "ejemplo", "catedra", "mendoza", "argentina", "muñoz", "facchini", "cesari", "xsd", "infoleg"]

for f in raw_files:
    print("\033[94mDetectando conceptos en archivo: " + f + "\033[0m")

    tokens = []

    with open(tokens_path + "/" + f + ".txt", "rb") as tf:
        tokens = tf.read().decode("utf-8").split("\n")

    for i in range(len(tokens) - window_size):
        window = tokens[i:i+window_size]

        def get_subarrays(arr):
            result = []
            n = len(arr)
            for r in range(1, n+1):  # sizes from 1 to n
                for indices in combinations(range(n), r):
                    subarray = [arr[i] for i in indices]
                    result.append(subarray)
            return result
        
        arrays = get_subarrays(window[1:])

        # arrays.insert(0, []) # Permite formar conceptos de una sola palabra

        for arr in arrays:
            arr.insert(0, window[0])
            arr = [s.lower() for s in arr]

            if any(re.search(r'(^[0-9\.\,]+$)|(-$)|(^.\.$)', s) for s in arr):
                continue

            if len(arr) != len(set(arr)):
                continue

            if any(ban in arr for ban in banned):
                continue

            arr.sort()
            
            t = tuple(arr)
            if (not t in related_table):
                related_table[t] = 0
            related_table[t] += 1

print("\033[92mCantidad de conceptos candidatos:" + str(len(related_table)) + "\033[0m")

### Creación Vocabulario
Teniendo los tokens conceptuales, creamos el vocabulario. Para crear el vocabulario, se realizará un filtro al diccionario `related_table` resultante de los pasos anteriores. Definimos los parámetros `min_freq` y `max_freq` para ello.

In [25]:
min_freq = 50
max_freq = 100

vocabulary = []

for tokens, freq in related_table.items():
    if (freq >= min_freq and freq <= max_freq):
        vocabulary.append(tokens)

concepts_file = "./data/vocabulary.txt"

with open(concepts_file, "wb") as cf:
    for concept in vocabulary:
        cf.write((str(concept) + "\n").encode("utf-8"))

print("\033[92mTamaño del vocabulario:" + str(len(vocabulary)) + "\033[0m")

En este punto, ya hemos seleccionado conjuntos de tokens que suelen aparecer cerca.

Estos conjuntos serán los conceptos, y pasarán a formar nuestro vocabulario.

Ahora, se debe tokenizar nuevamente los textos planos, utilizando los conceptos.

Para esto, se recorrerá cada archivo de /data/plain mediante una ventana deslizante del mismo tamaño utilizado para detectar conceptos, separando en palabras siempre y cuando no se encuentre dentro de la ventana las palabras de un concepto.

Se generarán nuevos tokens, siendo estos numéricos (/data/tokens_num). LLos positivos (o 0) corresponden al índice de un concepto en el vocabulario, mientras que los negativos indican la cantidad de tokens no reconocidos (\<unk\>). Esto se realizó de esta forma para ahorrar espacio y tiempo de procesamiento.

Si en una ventana se detectara más de un concepto, se agregarán todos los que se encuentre. Debido al procesamiento que se realizará más adelante, no debería importar el orden.

In [4]:
import ast

concepts_file = "./data/vocabulary.txt"
vocabulary = []

with open(concepts_file, "rb") as cf:
    lines = cf.read().decode("utf-8").split("\n")[:-1]
    vocabulary = [ast.literal_eval(l) for l in lines]

print("\033[92mTamaño del vocabulario:" + str(len(vocabulary)) + "\033[0m")

Tamaño del vocabulario:2726


In [9]:
import math


tokens_conceptos_path = "./data/tokens_conceptos"

metrics_2 = {}

window_size_concept_tokenization = window_size * 3

for f in raw_files:
    print("\033[Tokenizando archivo por conceptos: " + f + "\033[0m")
    metrics_2[f] = {}
    found_concepts = 0

    recent_concepts = {}

    with open(plain_path + "/" + f + ".txt", "rb") as pf:
        txt = pf.read().decode("utf-8")
        tokens = [t.text for t in esp.tokenizer(txt)]

        metrics_2[f]["tokens"] = len(tokens)

        with open(tokens_conceptos_path + "/" + f + ".txt", "wb") as tnf:

            unks = 0

            for i in range(len(tokens) - window_size_concept_tokenization):
                window = tokens[i:i+window_size_concept_tokenization]
                for k, v in recent_concepts.items():
                    if v > 0:
                        recent_concepts[k] -= 1

                unks += 1
            
                for ix, concept in enumerate(vocabulary):
                    if all(word in window for word in concept) and (not ix in recent_concepts or recent_concepts[ix] == 0):
                        tnf.write(("-" + str((unks-1)) + " " + str(ix) + " ").encode("utf-8"))
                        unks = 0
                        recent_concepts[ix] = window_size_concept_tokenization
                        found_concepts += 1

                    
    metrics_2[f]["concepts"] = found_concepts

okenizando archivo por conceptos: 3 - AP - AdProy_2_Trabajo en Equipo_2022
okenizando archivo por conceptos: 3 - AP - respuestas
okenizando archivo por conceptos: 3 - BD - caselli_manual-de-base-de-datos-2019
okenizando archivo por conceptos: 3 - BD - Guía 1
okenizando archivo por conceptos: 3 - BD - Guía 2
okenizando archivo por conceptos: 3 - CD - capitulo2
okenizando archivo por conceptos: 3 - CD - Comunicaciones y Redes de Computadores,7ma Edición - William Stallings
okenizando archivo por conceptos: 3 - CD - sistemas-de-comunicaciones-electronicas-tomasi-4ta-edicion
okenizando archivo por conceptos: 3 - DS - Actor. Definicion. Clasificacion (1)
okenizando archivo por conceptos: 3 - DS - Eje 1. Metodología y conceptos teóricos aplicados
okenizando archivo por conceptos: 3 - DS - Libro UML y Patrones - Larman
okenizando archivo por conceptos: 3 - DS - MerFNConceptos
okenizando archivo por conceptos: 4 - AS - Analisis PEST
okenizando archivo por conceptos: 4 - AS - Backups_raids
oken

In [10]:
print("\033[92m")
for archivo, item in metrics_2.items():
    print("Archivo " + archivo + ":")
    for nombre, valor in item.items():
        print("\t" + nombre + ": " + str(valor))
print("\033[0m")


Archivo 3 - AP - AdProy_2_Trabajo en Equipo_2022:
	tokens: 27662
	concepts: 275
Archivo 3 - AP - respuestas:
	tokens: 0
	concepts: 0
Archivo 3 - BD - caselli_manual-de-base-de-datos-2019:
	tokens: 28732
	concepts: 388
Archivo 3 - BD - Guía 1:
	tokens: 11365
	concepts: 390
Archivo 3 - BD - Guía 2:
	tokens: 9548
	concepts: 143
Archivo 3 - CD - capitulo2:
	tokens: 10412
	concepts: 236
Archivo 3 - CD - Comunicaciones y Redes de Computadores,7ma Edición - William Stallings:
	tokens: 455695
	concepts: 17102
Archivo 3 - CD - sistemas-de-comunicaciones-electronicas-tomasi-4ta-edicion:
	tokens: 530998
	concepts: 19677
Archivo 3 - DS - Actor. Definicion. Clasificacion (1):
	tokens: 1490
	concepts: 33
Archivo 3 - DS - Eje 1. Metodología y conceptos teóricos aplicados:
	tokens: 4415
	concepts: 41
Archivo 3 - DS - Libro UML y Patrones - Larman:
	tokens: 253057
	concepts: 3868
Archivo 3 - DS - MerFNConceptos:
	tokens: 2342
	concepts: 42
Archivo 4 - AS - Analisis PEST:
	tokens: 1004
	concepts: 4
Arc

# Creación dataset
A partir de los tokens numéricos, se iterará por cada secuencia de token con un nuevo tamaño de ventana, mayor, tratando de distinguir conceptos relacionados.

Esta ventana se centrará en cada token (no -1), almacenando en un diccionario el token central, los tokens en el contexto y ejemplos negativos (para evitar que la red neuronal, al entrenar, aprenda que todos los tokens siempre están relacionados).

In [7]:
!pip install torch

   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   ---------------------------------------- 1.8/216.1 MB 12.6 MB/s eta 0:00:18
   - -------------------------------------- 5.5/216.1 MB 16.0 MB/s eta 0:00:14
   - -------------------------------------- 8.9/216.1 MB 15.8 MB/s eta 0:00:14
   -- ------------------------------------- 13.6/216.1 MB 17.5 MB/s eta 0:00:12
   --- ------------------------------------ 18.1/216.1 MB 18.4 MB/s eta 0:00:11
   ---- ----------------------------------- 22.5/216.1 MB 18.8 MB/s eta 0:00:11
   ---- ----------------------------------- 26.0/216.1 MB 18.9 MB/s eta 0:00:11
   ----- ---------------------------------- 29.9/216.1 MB 18.8 MB/s eta 0:00:10
   ------ --------------------------------- 33.6/216.1 MB 18.4 MB/s eta 0:00:10
   ------- -------------------------------- 38.0/216.1 MB 18.6 MB/s eta 0:00:10
   ------- -------------------------------- 38.3/216.1 MB 18.6 MB/s eta 0:00:10
   ------- -------------------------------- 39.3/216


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import random
import math
import collections
import torch
import spacy

"""
def subsample(oraciones):
  # Subsample high-frequency words.
  # Comprueba que oraciones es una lista de listas
  if oraciones and isinstance(oraciones[0], list):
    #Transforma una lista anidada en una lista simple
    tokens = [token for line in oraciones for token in line]
  counter_obj = collections.Counter()
  counter_obj.update(tokens)
  num_tokens = sum(counter_obj.values())

  # Devuelve true si hay que conservar el token
  def keep(token):
      return(random.uniform(0, 1) <
              math.sqrt(1e-4 / counter_obj[token] * num_tokens))

  return ([[token for token in line if keep(token)] for line in oraciones],
          counter_obj)"""

In [ ]:
def get_centers_and_contexts(corpus, max_window_size):
  """Return center words and context words in skip-gram."""
  centers, contexts = [], []
  for line in corpus:
    # Para formar un par de "palabra central--palabra de contexto",
    # cada oración debe tener al menos 2 palabras
    if len(line) < 2:
      continue
    centers += line
    for i in range(len(line)):  # Ventana de contexto centrada en `i`
      window_size = random.randint(1, max_window_size)
      indices = list(range(max(0, i - window_size),
                            min(len(line), i + 1 + window_size)))
      # Excluir la palabra central de las palabras de contexto
      indices.remove(i)
      contexts.append([line[idx] for idx in indices])
  return centers, contexts

In [ ]:
class RandomGenerator:
  """Randomly draw among {1, ..., n} according to n sampling weights."""
  def __init__(self, sampling_weights):
    # Exclude
    self.population = list(range(1, len(sampling_weights) + 1))
    self.sampling_weights = sampling_weights
    self.candidates = []
    self.i = 0

  def draw(self):
    if self.i == len(self.candidates):
      # Cache `k` random sampling results
      self.candidates = random.choices(
          self.population, self.sampling_weights, k=10000)
      self.i = 0
    self.i += 1
    return self.candidates[self.i - 1]

In [ ]:
def get_negatives(all_contexts, vocab, counter, K):
    """Devuelve palabras ruidosas para muestreo negativo."""
    # Pesos de muestreo para palabras con índices 1, 2, ...
    # (índice 0 es el token <unk> excluido) en el vocabulario
    tokens = vocab.get_itos()
    sampling_weights = [counter[tokens[i]]**0.75
                        for i in range(1, len(tokens))]
    all_negatives, generator = [], RandomGenerator(sampling_weights)
    for contexts in all_contexts:
        negatives = []
        while len(negatives) < len(contexts) * K:
            neg = generator.draw()
            # Las palabras ruidosas no pueden ser de contexto
            if neg not in contexts:
                negatives.append(neg)
        all_negatives.append(negatives)
    return all_negatives

In [ ]:
def is_clean_token(token: "spacy.tokens.Token") -> bool:
    """Return True if the token should be kept for downstream processing."""
    return not (
        token.is_punct
        or token.is_space
        or token.is_stop
        or len(token.text) == 1
    )


def load_sentences_from_plain(
    plain_path,
    window_size,
    lowercase = True,
):
    
    sentences= []
    nlp = spacy.load("es_core_news_sm")
    raw_files = [os.path.splitext(f)[0] for f in listdir(plain_path) if isfile(join(plain_path, f))]
    for f in raw_files:
        with open(f"{plain_path}/{f}.txt", "rb") as pf:
            text = pf.read().decode("utf-8")
            doc = nlp(text)
            for sent in doc.sents:
                tokens = [tok.text for tok in sent if is_clean_token(tok)]
                if lowercase:
                    tokens = [t.lower() for t in tokens]
                if not tokens:
                    continue

                if window_size > 0:
                    # Break long sentences into fixed-size chunks
                    for i in range(0, len(tokens), window_size):
                        chunk = tokens[i : i + window_size]
                        if chunk:
                            sentences.append(chunk)
                else:
                    sentences.append(tokens)
        return sentences

In [ ]:
def collate_batch(data):
  max_len = max(len(c) + len(n) for _, c, n in data)
  centers, contexts_negatives, masks, labels = [], [], [], []
  for center, context, negative in data:
    centers += [center]
    cur_len = len(context) + len(negative)
    contexts_negatives += [context + negative + [0] * (max_len - cur_len)]
    masks += [[1] * cur_len + [0] * (max_len - cur_len)]
    labels += [[1] * len(context) + [0] * (max_len - len(context))]
  return (torch.tensor(centers).reshape((-1, 1)), torch.tensor(
        contexts_negatives), torch.tensor(masks), torch.tensor(labels))

In [ ]:
def load_data_isi(batch_size, max_window_size, num_noise_words):
    sentences = load_sentences_from_plain("./data/plain", window_size=max_window_size)
    sentences_ss, counter = subsample(sentences)
    corpus = sentences_ss            # here they’re already token lists
    all_centers, all_contexts = get_centers_and_contexts(corpus, max_window_size)
    all_negatives = get_negatives(all_contexts, vocabulary, counter, num_noise_words)

    class ISIDataset(torch.utils.data.Dataset):
        def __init__(self, centers, contexts, negatives):
            assert len(centers) == len(contexts) == len(negatives)
            self.centers = centers
            self.contexts = contexts
            self.negatives = negatives

        def __getitem__(self, index):
            return (self.centers[index], self.contexts[index],
                    self.negatives[index])

        def __len__(self):
            return len(self.centers)

    dataset = ISIDataset(all_centers, all_contexts, all_negatives)

    data_iter = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True,
                                      collate_fn=collate_batch)
    return data_iter, vocabulary

In [ ]:
data_iter, vocabulario = load_data_isi(512, 5, 5)
names = ['centers', 'contexts_negatives', 'lengths', 'labels']
for batch in data_iter:
    for name, data in zip(names, batch):
        print(name, 'shape:', data.shape)
    break

Hecho esto, podemos finalmente armar nuestro dataset. El mismo retornará (mediante get_item()) un centro, su contexto y sus ejemplos negativos.

A partir de este Dataset, a su vez, se generará un DataLoader.

In [15]:
import torch

NameError: name 'vocabulary' is not defined

A continuación, se armará la estructura de la red neuronal mediante skipgram, utilizando capas Embedding de pytorch.

Como función de pérdida, se utilizará entropía cruzada binaria (Sigmoidea). Esto es así pues requerimos clasificar dos conceptos según si están o no relacionado.

Se optó por abarcar todo el entrenamiento en una misma función. La misma incluye la inicialización de variables y el ciclo de entrenamiento en sí.

In [ ]:
# Load vocabulary from file
with open("./data/vocabulary.txt", "r", encoding="utf-8") as f:
    # Each line contains a word
    vocabulary = [line.strip() for line in f if line.strip()]

def tokenize_concepts(raw_path, plain_path, tokens_conceptos_path, vocabulary, window_size, esp):
    """
    Tokenize documents by identifying words from vocabulary using a sliding window approach.
    
    Args:
        raw_path (str): Path to raw documents
        plain_path (str): Path to plain text documents
        tokens_conceptos_path (str): Path to save concept tokens
        vocabulary (list): List of words to identify in the text
        window_size (int): Base window size (will be multiplied by 3)
        esp: spaCy model for Spanish tokenization
    
    Returns:
        dict: Metrics about tokens and concepts found per file
    """
    raw_files = [os.path.splitext(f)[0] for f in listdir(raw_path) if isfile(join(raw_path, f))]
    metrics = {}
    window_size_concept_tokenization = window_size * 3

    for f in raw_files:
        print("\033[94mTokenizando archivo por conceptos: " + f + "\033[0m")
        metrics[f] = {}
        found_concepts = 0
        recent_concepts = {}  # Track recently found concepts

        # Read and tokenize the plain text file
        with open(f"{plain_path}/{f}.txt", "rb") as pf:
            txt = pf.read().decode("utf-8")
            tokens = [t.text for t in esp.tokenizer(txt)]
            metrics[f]["tokens"] = len(tokens)

            # Write concept tokens to output file
            with open(f"{tokens_conceptos_path}/{f}.txt", "wb") as tnf:
                unks = 0  # Counter for unknown tokens

                for i in range(len(tokens) - window_size_concept_tokenization):
                    window = tokens[i:i+window_size_concept_tokenization]
                    
                    # Decrease counters for recent concepts and cleanup
                    for k, v in list(recent_concepts.items()):
                        if v > 0:
                            recent_concepts[k] -= 1
                        if recent_concepts[k] == 0:
                            del recent_concepts[k]

                    unks += 1

                    # Check each word in vocabulary against the current window
                    for ix, word in enumerate(vocabulary):
                        # If the word is in the window and not recently found
                        if word in window and ix not in recent_concepts:
                            tnf.write(f"-{unks-1} {ix} ".encode("utf-8"))
                            unks = 0
                            recent_concepts[ix] = window_size_concept_tokenization
                            found_concepts += 1

            metrics[f]["concepts"] = found_concepts

    return metrics

# Run concept tokenization
metrics = tokenize_concepts(
    raw_path=raw_path,
    plain_path=plain_path,
    tokens_conceptos_path=tokens_conceptos_path,
    vocabulary=vocabulary,
    window_size=5,
    esp=esp
)

# Print metrics
for file, file_metrics in metrics.items():
    print(f"\nFile: {file}")
    print(f"Total tokens: {file_metrics['tokens']}")
    print(f"Concepts found: {file_metrics['concepts']}")


In [ ]:
def tokenize_concepts(raw_path, plain_path, tokens_conceptos_path, vocabulary, window_size, esp):
    """
    Tokenize documents by identifying pre-defined concepts using a sliding window approach.
    
    Args:
        raw_path (str): Path to raw documents
        plain_path (str): Path to plain text documents
        tokens_conceptos_path (str): Path to save concept tokens
        vocabulary (list): List of concepts to identify
        window_size (int): Base window size (will be multiplied by 3)
        esp: spaCy model for Spanish tokenization
    
    Returns:
        dict: Metrics about tokens and concepts found per file
    """
    raw_files = [os.path.splitext(f)[0] for f in listdir(raw_path) if isfile(join(raw_path, f))]
    metrics = {}
    window_size_concept_tokenization = window_size * 3

    for f in raw_files:
        print("\033[94mTokenizando archivo por conceptos: " + f + "\033[0m")
        metrics[f] = {}
        found_concepts = 0
        recent_concepts = {}  # Track recently found concepts

        # Read and tokenize the plain text file
        with open(f"{plain_path}/{f}.txt", "rb") as pf:
            txt = pf.read().decode("utf-8")
            tokens = [t.text for t in esp.tokenizer(txt)]
            metrics[f]["tokens"] = len(tokens)

            # Write concept tokens to output file
            with open(f"{tokens_conceptos_path}/{f}.txt", "wb") as tnf:
                unks = 0  # Counter for unknown tokens

                for i in range(len(tokens) - window_size_concept_tokenization):
                    window = tokens[i:i+window_size_concept_tokenization]
                    
                    # Decrease counters for recent concepts and cleanup
                    for k, v in list(recent_concepts.items()):
                        if v > 0:
                            recent_concepts[k] -= 1
                        if recent_concepts[k] == 0:
                            del recent_concepts[k]

                    unks += 1

                    # Check each concept against the current window
                    for ix, concept in enumerate(vocabulary):
                        # If all words of the concept are in the window and concept not recently found
                        if all(word in window for word in concept) and ix not in recent_concepts:
                            tnf.write(f"-{unks-1} {ix} ".encode("utf-8"))
                            unks = 0
                            recent_concepts[ix] = window_size_concept_tokenization
                            found_concepts += 1

            metrics[f]["concepts"] = found_concepts

    return metrics


In [ ]:
# Load dependencies
import spacy
import os
from os import listdir
from os.path import isfile, join

# Load Spanish language model
esp = spacy.load("es_core_news_sm")

# Paths
raw_path = "./data/raw"
plain_path = "./data/plain"
tokens_path = "./data/tokens"
tokens_conceptos_path = "./data/tokens_conceptos"
os.makedirs(tokens_conceptos_path, exist_ok=True)

# Create vocabulary from tokens with frequency filtering
vocabulary = Vocab(token_iterator(tokens_path), min_freq=50, max_freq=100)

# Convert vocabulary to list of tuples for concept detection
# Each concept is a single word in this case, but we keep it as a tuple for consistency
vocab_concepts = [(token,) for token in vocabulary.idx_to_token if token != '<unk>']
print("\033[92mTamaño del vocabulario: " + str(len(vocab_concepts)) + "\033[0m")

# Run concept tokenization
metrics = tokenize_concepts(
    raw_path=raw_path,
    plain_path=plain_path,
    tokens_conceptos_path=tokens_conceptos_path,
    vocabulary=vocab_concepts,  # Pass the list of tuples instead of Vocab object
    window_size=5,
    esp=esp
)

# Print metrics
for file, file_metrics in metrics.items():
    print(f"\nFile: {file}")
    print(f"Total tokens: {file_metrics['tokens']}")
    print(f"Concepts found: {file_metrics['concepts']}")


In [ ]:
# Example usage:
import os
from os import listdir
from os.path import isfile, join
import spacy

# Load Spanish language model
esp = spacy.load("es_core_news_sm")

# Paths
raw_path = "./data/raw"
plain_path = "./data/plain"
tokens_conceptos_path = "./data/tokens_conceptos"
os.makedirs(tokens_conceptos_path, exist_ok=True)

# Load vocabulary from file
with open("./data/vocabulary.txt", "r", encoding="utf-8") as f:
    # Each line contains a tuple of words that form a concept
    vocabulary = [eval(line.strip()) for line in f if line.strip()]

# Run concept tokenization
metrics = tokenize_concepts(
    raw_path=raw_path,
    plain_path=plain_path,
    tokens_conceptos_path=tokens_conceptos_path,
    vocabulary=vocabulary,
    window_size=5,
    esp=esp
)

# Print metrics
for file, file_metrics in metrics.items():
    print(f"\nFile: {file}")
    print(f"Total tokens: {file_metrics['tokens']}")
    print(f"Concepts found: {file_metrics['concepts']}")


Se generó una función auxiliar para el entrenamiento por medio de GPU, en caso de estar disponible.

In [ ]:
def try_gpu(i=0):
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

Finalmente, se realizó el entrenamiento:

Para verificar la funcionalidad final que buscamos en el proyecto, se planteó la siguiente función:

Algunos ejemplos de la misma serían: